# 단순한 Convolutional Neural Networks 실습 (합성곱 신경망)

다음 코드는 기본적인 컨브넷의 모습입니다. 
`Conv2D`와 `MaxPooling2D` 층을 쌓아 올렸습니다. 

컨브넷이 `(image_height, image_width, image_channels)` 크기의 입력 텐서를 사용한다는 점이 중요합니다(배치 차원은 포함하지 않습니다).

이 예제에서는 MNIST 이미지 포맷인 `(28, 28, 1)` 크기의 입력을 처리하도록 컨브넷을 설정해야 합니다. 이 때문에 첫 번째 층의 매개변수로 `input_shape=(28, 28, 1)`을 전달합니다.

### 코딩 시작 : 주석을 이용하여 None을 채우세요

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential()

# 32개의 3x3 conv. filter, activation은 relu, input_shape은 (28, 28, 1)
model.add(layers.Conv2D(None, (3,3), activation='relut', input_shape=(28,28,1)))

# 2x2, stride 2의 maxpooling
model.add(layers.MaxPooling2D())

# 64개의 3x3 conv. filter, activation은 relu
model.add(layers.Conv2D(64, (3,3), activation='relu'))

# 2x2, stride 2의 maxpooling
model.add(layers.MaxPooling2D())

# 64개의 3x3 conv. filter, activation은 relu
model.add(layers.Conv2D(64, (3,3), activation=None))

지금까지 컨브넷 구조를 출력해 보죠:

In [ ]:
model.summary()

`Conv2D`와 `MaxPooling2D` 층의 출력은 `(height, width, channels)` 크기의 3D 텐서입니다. 높이와 넓이 차원은 네트워크가 깊어질수록 작아지는 경향이 있습니다. 채널의 수는 `Conv2D` 층에 전달된 첫 번째 매개변수에 의해 조절됩니다(32개 또는 64개).

다음 단계에서 마지막 층의 (`(3, 3, 64)` 크기인) 출력 텐서를 완전 연결 네트워크에 주입합니다. 이 네트워크는 이미 익숙하게 보았던 `Dense` 층을 쌓은 분류기입니다. 이 분류기는 1D 벡터를 처리하는데 이전 층의 출력이 3D 텐서입니다. 그래서 먼저 3D 출력을 1D 텐서로 펼쳐야 합니다. 그다음 몇 개의 `Dense` 층을 추가합니다:

### 코딩 시작 : 주석을 이용하여 None을 채우세요

In [ ]:
model.add(layers.Flatten())

# 64개의 Unit추가, activation은 relu
model.add(layers.Dense(64, activation='relu'))

# 10개 (클래스의 수)의 Unit추가, activation은 softmax
model.add(layers.Dense(10, activation=None))

10개의 클래스를 분류하기 위해 마지막 층의 출력 크기를 10으로 하고 소프트맥스 활성화 함수를 사용합니다. 이제 전체 네트워크는 다음과 같습니다:

In [ ]:
model.summary()

여기에서 볼 수 있듯이 `(3, 3, 64)` 출력이 `(576,)` 크기의 벡터로 펼쳐진 후 `Dense` 층으로 주입되었습니다.

이제 MNIST 숫자 이미지에 이 컨브넷을 훈련합니다. 

In [ ]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [ ]:
import matplotlib.pyplot as plt
train_images[0].shape
plt.imshow(train_images[0].reshape(28,28))
print(train_labels[0])

In [ ]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

테스트 데이터에서 모델을 평가해 보죠:

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

In [ ]:
test_acc

2장의 완전 연결 네트워크는 97.8%의 테스트 정확도를 얻은 반면, 기본적인 컨브넷은 99.2%의 테스트 정확도를 얻었습니다.